In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/Shared drives/stand_mapping')

In [3]:
! sudo apt-get install -y libspatialindex-dev
# this is a requirement for the rtree package, which is used by geopandas to calculate intersections
# if you conda install (instead of pip install) geopandas, you should not need to install this package or rtree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [0]:
! pip install rasterio geopandas rtree -q

In [0]:
# install our stand_mapping package
! python setup.py develop

running develop
running egg_info
writing stand_mapping.egg-info/PKG-INFO
writing dependency_links to stand_mapping.egg-info/dependency_links.txt
writing top-level names to stand_mapping.egg-info/top_level.txt
writing manifest file 'stand_mapping.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.6/dist-packages/stand-mapping.egg-link (link to .)
stand-mapping 0.1.0 is already the active version in easy-install.pth

Installed /content/drive/Shared drives/stand_mapping
Processing dependencies for stand-mapping==0.1.0
Finished processing dependencies for stand-mapping==0.1.0


In [0]:
import geopandas as gpd
from matplotlib import pyplot as plt
import matplotlib.patheffects as pe
from rasterio.plot import show, reshape_as_raster
from rasterio.transform import from_bounds
from shapely.geometry import box
from stand_mapping.data import fetch
%matplotlib inline

# Read in some vector data
This layer shows the forest stands at the University of Washington's Charles L. Pack Experimental Forest.

In [0]:
PACK_SHP = '/content/drive/Shared drives/stand_mapping/data/external/packforest/packforest_stands2015_epsg26910.shp'
pack_stands = gpd.read_file(PACK_SHP)
pack_stands['geometry'] = pack_stands.buffer(0)
pack_stands.plot();

Define our Area of Interest with a Bounding Box

In [0]:
XMIN, YMIN, XMAX, YMAX = 553915, 5184662,  555451, 5186198
EPSG = 26910  # specify the coordinate system, this is UTM 10N

In [0]:
# define the dimensions of the image you want retrieved
WIDTH, HEIGHT = 512, 512  # downsampling will be handled server-side

# alternatively, you can request at full resolution
# WIDTH, HEIGHT = XMAX - XMIN, YMAX - YMIN

In [0]:
bbox = (XMIN, YMIN, XMAX, YMAX)

Fetch NAIP Aerial Imagery from The National Map REST Service

In [0]:
naip = fetch.naip_from_tnm(bbox=bbox, bboxSR=EPSG, size=(WIDTH,HEIGHT), imageSR=EPSG, format='jpgpng')
naip.shape

# Quick View of the Aerial Imagery
Note that extent uses local image coordinates. There is no awareness of geospatial coordinates yet.

In [0]:
plt.imshow(naip);

# Use Rasterio to Georeference and Plot
We first need to convert the image into the shape that `rasterio` wants. The TNM NAIP REST web service returns the RGB image with shape `(width, height, channels)`. `rasterio`, however, expects a raster format with shape `(bands, width, height`). 

Once we calculate the affine transformation that will project the image into the appropiate geographic extent, we can display the raster in the appropriate geographic extent using `rasterio.plot.show`.



In [0]:
affine_transform = from_bounds(*bbox, HEIGHT, WIDTH)
# use rasterio.plot.show to pull this all together

raster = reshape_as_raster(naip)
show(raster, transform=affine_transform);

In [0]:
raster.shape

# Add Georeferenced Overlays
Now that the extent of our image is related to the geographic coordinates, we can also add vectors to the display.

In [0]:
fig, ax = plt.subplots(figsize=(10,10))

show(raster, transform=affine_transform, ax=ax)
clipped_stands = gpd.clip(pack_stands, 
                          # generate a bbox polygon using shapely's box
                          mask=box(*bbox).buffer(2) # buffer slightly so that edges are off the image
                          )
clipped_stands.plot(ax=ax, facecolor='none', edgecolor='yellow')
ax.axis('off')
plt.show();

# Add some labels to the polygons

In [0]:
clipped_stands['coords'] = clipped_stands['geometry'].apply(lambda x: x.representative_point().coords[0])

fig, ax = plt.subplots(figsize=(10,10))

show(raster, transform=affine_transform, ax=ax)
clipped_stands.plot(ax=ax, facecolor='none', edgecolor='orange', lw=1.5, alpha=0.5)

for idx, row in clipped_stands.iterrows():
    label_x, label_y = row['coords']
    label = idx
    
    # this block moves labels away from edges
    if abs(label_x - XMIN) < 10:
        horizontalalignment='left'
        label_x += 15
    if abs(label_x - XMAX) < 10:
        horizontalalignment='right'
        label_x -= 15
    else:
        horizontalalignment='center'
    if abs(label_y - YMIN) < 10:
        verticalalignment='top'
        label_y += 15
    elif abs(label_y - YMAX) < 10:
        verticalalignment='bottom'
        label_y -= 15
    else:
        verticalalignment='center'
    
    # add the labels
    ax.annotate(s=label, 
                xy=(label_x,label_y),
                horizontalalignment=horizontalalignment,
                verticalalignment=verticalalignment,
                color='orange', size=12, fontweight='bold',
                path_effects=[pe.withStroke(linewidth=3, foreground='black')])
    
ax.axis('off')
plt.show();